# Limpieza de datos

## Acerca de los datos
En este notebook, utilizaremos datos diarios de temperatura de la API [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). Utilizaremos el conjunto de datos Global Historical Climatology Network - Daily (GHCND); consulte la documentación [aquí](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

Estos datos se recogieron en la estación del aeropuerto LaGuardia de Nueva York para octubre de 2018. Contiene:
- la temperatura mínima diaria (`TMIN`)
- la temperatura máxima diaria (`TMAX`)
- la temperatura media diaria (`TAVG`)

*Nota: El NCEI forma parte de la Administración Nacional Oceánica y Atmosférica (NOAA) y, como puede ver en la URL de la API, este recurso se creó cuando el NCEI se llamaba NCDC. Si la URL de este recurso cambiara en el futuro, puede buscar "NCEI weather API" para encontrar la actualizada.*

Además, utilizaremos datos del mercado de valores S&P 500 (obtenidos mediante el paquete [`stock_analysis`](https://github.com/stefmolin/stock-analysis) que construiremos en el capítulo 7) y datos para bitcoin de 2017 a 2018. En la primera edición, los datos de bitcoin se recopilaron de CoinMarketCap utilizando el paquete `stock_analysis`; sin embargo, los cambios en el sitio web obligaron a cambiar la fuente de datos a Yahoo! Finance. Los datos de bitcoin que se recopilaron antes del cambio de sitio web de CoinMarketCap deberían ser equivalentes a los datos históricos que se pueden ver en [esta](https://coinmarketcap.com/currencies/bitcoin/historical-data/) página.

## Configuración
Necesitamos importar `pandas` y leer los datos de temperatura para empezar:

In [ ]:
import pandas as pd

df = pd.read_csv('data/nyc_temperatures.csv')
df.head()

## Renombrar Columnas
Comenzamos con las siguientes columnas:

In [ ]:
df.columns

Queremos renombrar la columna `value` para indicar que contiene la temperatura en grados Celsius y la columna `attributes` para decir `flags` ya que cada valor de la cadena delimitada por comas es una bandera diferente sobre la recogida de datos. Para esta tarea, utilizamos el método `rename()` y pasamos un diccionario que asigna los nombres de las columnas a sus nuevos nombres. Pasamos `inplace=True` para cambiar nuestro marco de datos original en lugar de obtener uno nuevo:

In [ ]:
df.rename(
    columns={
        'value': 'temp_C',
        'attributes': 'flags'
    }, inplace=True
)

Estas columnas se han renombrado correctamente:

In [ ]:
df.columns

También podemos realizar operaciones de cadena sobre los nombres de las columnas con `rename()`:

In [ ]:
df.rename(str.upper, axis='columns').columns

## Conversión de tipos
La columna `date` no se almacena actualmente como `datetime`:

In [ ]:
df.dtypes

Realicemos la conversión con `pd.to_datetime()`:

In [ ]:
df.loc[:,'date'] = pd.to_datetime(df.date)
df.dtypes

Ahora obtendremos información útil cuando utilicemos `describe()` en esta columna:

In [ ]:
df.date.describe()

Podemos utilizar `tz_localize()` en un objeto `DatetimeIndex` para convertirlo a la zona horaria deseada:

In [ ]:
pd.date_range(start='2018-10-25', periods=2, freq='D').tz_localize('EST')

Esto también funciona con objetos `Series`/`DataFrame` que tengan un índice de tipo `DatetimeIndex`. Vamos a leer en el CSV de nuevo para este ejemplo y establecer la columna "date" para ser el índice y se almacena como una fecha y hora:

In [ ]:
eastern = pd.read_csv(
    'data/nyc_temperatures.csv', index_col='date', parse_dates=True
).tz_localize('EST')
eastern.head()

Podemos usar `tz_convert()` para convertir a otra zona horaria desde ahí. Si convertimos las fechas de Eastern a UTC, ahora serán a las 5 AM, ya que `pandas` utilizará los desfases para convertir:

In [ ]:
eastern.tz_convert('UTC').head()

También podemos cambiar el periodo del índice. Podríamos cambiar el periodo a mensual para facilitar la agregación posterior. (La agregación se tratará en el capítulo 4).

In [ ]:
eastern.tz_localize(None).to_period('M').index

Ahora obtenemos un objeto `PeriodIndex`, que podemos volver a convertir en un objeto `DatetimeIndex` con `to_timestamp()`:

In [ ]:
eastern.tz_localize(None).to_period('M').to_timestamp().index

Podemos utilizar el método `assign()` para trabajar con varias columnas a la vez (o crear nuevas). Como nuestra columna `date` ya ha sido convertida, necesitamos leer los datos de nuevo:

In [ ]:
df = pd.read_csv('data/nyc_temperatures.csv').rename(
    columns={
        'value': 'temp_C',
        'attributes': 'flags'
    }
)

new_df = df.assign(
    date=pd.to_datetime(df.date),
    temp_F=(df.temp_C * 9/5) + 32
)
new_df.dtypes

La columna `date` tiene ahora fecha y hora y se ha añadido la columna `temp_F`:

In [ ]:
new_df.head()

También podemos utilizar `astype()` para realizar conversiones. Vamos a crear columnas de la parte entera de las temperaturas en Celsius y Fahrenheit. Usaremos **funciones lambda** (introducidas por primera vez en *Capítulo 2, Trabajando con Pandas DataFrames*), de forma que podamos usar los valores que se están creando en la columna `temp_F` para calcular la columna `temp_F_entera`. Es muy común (y útil) usar funciones lambda con `assign()`:

In [ ]:
df = df.assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_C_whole=lambda x: x.temp_C.astype('int'),
    temp_F=lambda x: (x.temp_C * 9/5) + 32,
    temp_F_whole=lambda x: x.temp_F.astype('int')
)

df.head()

Creación de categorías:

In [ ]:
df_with_categories = df.assign(
    station=df.station.astype('category'),
    datatype=df.datatype.astype('category')
)
df_with_categories.dtypes

In [ ]:
df_with_categories.describe(include='category')

Nuestras categorías no tienen orden, pero esto es algo que soporta `pandas`:

In [ ]:
pd.Categorical(
    ['med', 'med', 'low', 'high'], 
    categories=['low', 'med', 'high'],
    ordered=True
)

## Reordenación, reindexación y ordenación
Digamos que queremos encontrar los días que alcanzaron las temperaturas más altas en los datos meteorológicos; podemos ordenar nuestros valores por la columna `temp_C` con la mayor en la parte superior para encontrar esto:

In [ ]:
df[df.datatype == 'TMAX'].sort_values(by='temp_C', ascending=False).head(10)

Sin embargo, esto no es perfecto porque tenemos algunos empates, y no están ordenados de forma coherente. En el primer empate entre la 7ª y la 10ª, la fecha anterior aparece en primer lugar, pero ocurre lo contrario con el empate entre la 4ª y la 2ª. Podemos utilizar otras columnas para desempatar y especificar cómo ordenar cada una con `ascendente`. Rompamos los empates con la columna de fecha y mostremos las fechas anteriores antes que las posteriores:

In [ ]:
df[df.datatype == 'TMAX'].sort_values(by=['temp_C', 'date'], ascending=[False, True]).head(10)

Observe que el índice estaba desordenado en los 2 últimos resultados. En este caso, nuestro índice sólo almacena el número de fila en los datos originales, pero puede que no necesitemos hacer un seguimiento de esa información. En este caso, podemos pasar `ignore_index=True` para obtener un nuevo índice después de ordenar:

In [ ]:
df[df.datatype == 'TMAX'].sort_values(by=['temp_C', 'date'], ascending=[False, True], ignore_index=True).head(10)

Cuando sólo se buscan los n valores más grandes, en lugar de querer ordenar todos los datos, podemos utilizar `nlargest()`:

In [ ]:
df[df.datatype == 'TAVG'].nlargest(n=10, columns='temp_C')

Utilizamos `nsmallest()` para los n valores más pequeños.

In [ ]:
df.nsmallest(n=5, columns=['temp_C', 'date'])

El método `sample()` nos dará filas (o columnas con `axis=1`) al azar. Podemos proporcionar una semilla (`estado_aleatorio`) para hacer esto reproducible. El índice después de hacer esto está desordenado:

In [ ]:
df.sample(5, random_state=0).index

Podemos usar `sort_index()` para ordenarlo de nuevo:

In [ ]:
df.sample(5, random_state=0).sort_index().index

El método `sort_index()` también puede ordenar las columnas alfabéticamente:

In [ ]:
df.sort_index(axis=1).head()

Esto puede facilitar la selección con `loc` para muchas columnas:

In [ ]:
df.sort_index(axis=1).head().loc[:,'temp_C':'temp_F_whole']

Debemos ordenar el índice para comparar dos marcos de datos. Si el índice es diferente, pero los datos son los mismos, se marcarán como no iguales:

In [ ]:
df.equals(df.sort_values(by='temp_C'))

La ordenación del índice resuelve este problema:

In [ ]:
df.equals(df.sort_values(by='temp_C').sort_index())

Fijemos la columna `date` como nuestro índice:

In [ ]:
df.set_index('date', inplace=True)
df.head()

Ahora que tenemos un índice de tipo `DatetimeIndex`, podemos hacer el corte y la indexación datetime. Siempre que proporcionemos un formato de fecha que pandas entienda, podremos obtener los datos. Para seleccionar todo 2018, simplemente usamos `df.loc['2018']`, para el cuarto trimestre de 2018 podemos usar `df.loc['2018-Q4']`, coger octubre es tan simple como usar `df.loc['2018-10']`; estos también se pueden combinar para construir rangos. Tomemos desde el 11 de octubre de 2018 hasta el 12 de octubre de 2018 (incluidos ambos puntos finales)&mdash;tenga en cuenta que el uso de `loc[]` es opcional para los rangos:

In [ ]:
df['2018-10-11':'2018-10-12']

También podemos usar `reset_index()` para obtener un índice nuevo y mover nuestro índice actual a una columna para guardarlo. Esto es especialmente útil si teníamos datos, como la fecha, en el índice que no queremos perder:

In [ ]:
df['2018-10-11':'2018-10-12'].reset_index()

La reindexación nos permite conformar nuestro eje para que contenga un conjunto determinado de etiquetas. Veamos un ejemplo con los datos de las acciones del S&P 500 en el archivo `sp500.csv`. Observe que sólo tenemos datos de los días de negociación (días laborables, excepto festivos):

In [ ]:
sp = pd.read_csv(
    'data/sp500.csv', index_col='date', parse_dates=True
).drop(columns=['adj_close'])

sp.head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

Si queremos analizar el valor de una cartera (grupo de activos) que cotiza en días distintos, tenemos que gestionar el desajuste en el índice. Bitcoin, por ejemplo, cotiza diariamente. Si sumamos todos los datos que tenemos para cada día (las agregaciones se tratarán en el capítulo 4, así que no te fijes en esta parte), obtenemos lo siguiente:

In [ ]:
bitcoin = pd.read_csv(
    'data/bitcoin.csv', index_col='date', parse_dates=True
).drop(columns=['market_cap'])

# precio de cierre de cada día = cierre del S&P 500 + cierre del Bitcoin (lo mismo para otras métricas)
portfolio = pd.concat([sp, bitcoin], sort=False).groupby(level='date').sum()

portfolio.head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

Puede que no sea inmediatamente obvio lo que pasa con los datos anteriores, pero con una visualización podemos ver fácilmente el patrón cíclico de caídas en los días en que la bolsa está cerrada. (No te preocupes demasiado por el código de trazado, lo trataremos en profundidad en los capítulos 5 y 6).

Ahora necesitaremos importar `matplotlib`:

In [ ]:
import matplotlib.pyplot as plt # usamos este módulo para trazar
from matplotlib.ticker import StrMethodFormatter # para formatear el eje

Now we can see why we need to reindex:

In [ ]:
# traza el precio de cierre desde el cuarto trimestre de 2017 hasta el segundo trimestre de 2018
ax = portfolio['2017-Q4':'2018-Q2'].plot(
    y='close', figsize=(15, 5), legend=False,
    title="Bitcoin + valor del S&P 500 sin tener en cuenta los distintos índices"
)

# formato
ax.set_ylabel('price')
ax.yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

# mostrar el plot
plt.show()

Necesitamos alinear el índice del S&P 500 para que coincida con el bitcoin para solucionar esto. Usaremos el método `reindex()`, pero por defecto obtenemos `NaN` para los valores de los que no tenemos datos:

In [ ]:
sp.reindex(bitcoin.index).head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

Así que ahora tenemos filas para cada día del año, pero todos los fines de semana y días festivos tienen valores `NaN`. Para solucionar esto, podemos especificar cómo tratar los valores que faltan con el argumento `method`. En este caso, lo que queremos es `forward-fill`, que pondrá los valores de los fines de semana y festivos como el valor que tenían el viernes (o al final de la semana comercial) anterior:

In [ ]:
sp.reindex(bitcoin.index, method='ffill').head(10)\
    .assign(day_of_week=lambda x: x.index.day_name())

Para aislar los cambios que se producen con el relleno hacia delante, podemos utilizar el método `compare()`. Nos muestra los valores que difieren entre marcos de datos etiquetados de forma idéntica (mismos nombres y mismas columnas). Aquí, podemos ver que solo los fines de semana y los días festivos (el lunes 16 de enero de 2017 fue el día de MLK) tienen valores rellenados hacia adelante. Observe que los días consecutivos tienen los mismos valores.

In [ ]:
sp.reindex(bitcoin.index)\
    .compare(sp.reindex(bitcoin.index, method='ffill'))\
    .head(10).assign(day_of_week=lambda x: x.index.day_name())

Sin embargo, esto no es perfecto. Probablemente queramos 0 para el volumen negociado y poner el precio de cierre para la apertura, el máximo, el mínimo y el cierre en los días en que el mercado está cerrado:

In [ ]:
import numpy as np

sp_reindexed = sp.reindex(bitcoin.index).assign(
    volume=lambda x: x.volume.fillna(0), # poner 0 cuando el mercado está cerrado
    close=lambda x: x.close.fillna(method='ffill'), # llevar esto adelante
    # tomar el precio de cierre si no están disponibles
    open=lambda x: np.where(x.open.isnull(), x.close, x.open),
    high=lambda x: np.where(x.high.isnull(), x.close, x.high),
    low=lambda x: np.where(x.low.isnull(), x.close, x.low)
)
sp_reindexed.head(10).assign(
    day_of_week=lambda x: x.index.day_name()
)

Si creamos una visualización comparando los datos reindexados con el primer intento, vemos cómo la reindexación ayudó a mantener el valor del activo cuando el mercado estaba cerrado:

In [ ]:
# precio de cierre de cada día = cierre del S&P 500 ajustado al cierre del mercado + cierre del Bitcoin (lo mismo para otras métricas)
fixed_portfolio = sp_reindexed + bitcoin

# Traza el precio de cierre de la cartera reindexada desde el cuarto trimestre de 2017 hasta el segundo trimestre de 2018.
ax = fixed_portfolio['2017-Q4':'2018-Q2'].plot(
    y='close', label='reindexed portfolio of S&P 500 + Bitcoin', figsize=(15, 5), linewidth=2, 
    title='Reindexed portfolio vs. portfolio with mismatched indices'
)

# añadir línea para la cartera original para comparación
portfolio['2017-Q4':'2018-Q2'].plot(
    y='close', ax=ax, linestyle='--', label='portfolio of S&P 500 + Bitcoin w/o reindexing'
)

# formateo
ax.set_ylabel('price')
ax.yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

# mostrar el plot
plt.show()

<hr>
<div>
    <a href="./2-usando_la_api_weather.ipynb">
        <button>&#8592; Notebook Anterior</button>
    </a>
    <a href="./4-reshaping_data.ipynb">
        <button style="float: right;"> Siguiente Notebook &#8594;</button>
    </a>
</div>
<hr>